# Import library yang digunakan

In [ ]:
!pip install googletrans==4.0.0rc1
!pip install textblob

In [ ]:
from collections import defaultdict
from textblob import TextBlob
from googletrans import Translator
import json

# Sentiment Analysis

## Persiapan model

In [ ]:
# Inisialisasi translator untuk review dengan bahasa selain Inggris
translator = Translator()

# file JSON untuk menyimpan data review agar tetap tersimpan walau keluar dari menu
DATA_FILE = "reviews_data.json"

# Data untuk menyimpan review berdasarkan tempat
places_reviews = {}
# Data untuk menyimpan jumlah review per sentimen untuk setiap tempat
sentiment_summary = defaultdict(lambda: {"Positive": 0, "Netral": 0, "Negative": 0})

## Fungsi sentiment analysis menggunakan Lexical NLP dengan textblob

In [ ]:
#Membuat fungsi analisis sentiment
def analyze_sentiment(review):
    """
    Fungsi untuk menganalisis sentimen menggunakan TextBlob (Lexical NLP).
    Input: Teks review yang akan dianalisis.
    Output: Jenis review tersebut 'Positive', 'Netral', atau 'Negative'.
    """
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    #Menentukan sentiment berdasarkan nilai polaritas
    if polarity > 0.1:
        sentiment = 'Positive'
    elif polarity < -0.1:
        sentiment = 'Negative'
    else:
        sentiment = 'Netral'

    return sentiment

## Fungsi untuk menerjemahkan review dengan bahasa selain Inggris

In [ ]:
#Membuat fungsi yang menerjermahkan review
def translate_review(review, src_lang='auto'):
    """
    Fungsi untuk menerjemahkan review ke bahasa Inggris menggunakan Google Translate (TextBlob hanya bisa membaca teks bahasa Inggris)
    Review: Teks review yang akan diterjemahkan.
    Src_lang: Bahasa yang digunakan di review. Dipilih auto agar dapat menerjemahkan bahasa apapun yang terdeteksi.
    """
    try:
        translation = translator.translate(review, src=src_lang, dest='en') # diterjemahkan ke bahasa Inggris
        return translation.text
    except Exception as e:
        print(f"Error occured when translating: {e}") # Jika bahasa tidak diketahui google translate
        return review

## Fungsi untuk memuat file JSON yang akan digunakan untuk penyimpanan data review

In [ ]:
#Membuat fungsi untuk menyimpan review
def load_data():
    """
    Fungsi untuk memuat file JSON yang digunakan dalam penyimpanan data review.
    """
    global places_reviews, sentiment_summary
    try:
        with open(DATA_FILE, "r") as file:
            data = json.load(file)
            # Mengambil  data review tempat dan rangkuman sentimen
            places_reviews = data["places_reviews"]
            sentiment_summary = defaultdict(lambda: {"Positive": 0, "Netral": 0, "Negative": 0}, data["sentiment_summary"])
            print("Data loaded successfully.")
    except FileNotFoundError:
        print("No previous data found, starting from scratch.")
    except json.JSONDecodeError:
        print("Corrupted data files. Starting with empty data.")

## Fungsi untuk menyimpan data review ke file JSON

In [ ]:
#Membuat fungsi untuk menyimpan data review ke file JSON
def save_data():
    """
    Fungsi untuk menyimpan data review ke file JSON.
    """
    #Membuka file dengan setting write
    with open(DATA_FILE, "w") as file:
      # Menyiapkan data yang akan disimpan
        data = {
            "places_reviews": places_reviews,
            "sentiment_summary": sentiment_summary,
        }
        #Menyimpan data ke file json
        json.dump(data, file, indent=4)
        print("Data has been saved.")


## Fungsi untuk reset history review

In [ ]:
#Membuat fungsi untuk mereset history data review
def reset_history():
    """
    Fungsi untuk mereset history data review.
    """
    global places_reviews, sentiment_summary
    # Konfirmasi dari user apakah ingin menghapus history review
    confirm = input("Are you sure want to delete all review data history? (y/n): ").strip().lower()
    if confirm == 'y':
        places_reviews = {}
        sentiment_summary = defaultdict(lambda: {"Positive": 0, "Netral": 0, "Negative": 0})
        save_data()  # Simpan data kosong ke file JSON
        print("All review data history have been deleted.")
    else:
        print("History reset cancelled.")


## Fungsi untuk menambahkan data review dari input user ke dictionary

In [ ]:
#Membuat fungsi untuk menambahkan data review
def add_review_interactive():
    """
    Fungsi untuk menu menambahkan review melalui input user.
    """
    # Menerima input nama tempat dan review dari pengguna
    place = input("Enter the place name: ").strip()
    review = input("Enter your review about the place (can be in any language): ").strip()

    translated_review = translate_review(review) # Menerjemahkan terlebih dahulu ke bahasa Inggris
    sentiment = analyze_sentiment(translated_review) # Setelah diterjemahkan barulah dilakukan sentiment analysis

    #Menambahkan tempat jika belum ada di tempat yang sudah di review
    if place not in places_reviews:
        places_reviews[place] = []

    #Menambahkan revie dan analisis sentiment sesuai tempatnya
    places_reviews[place].append({'review': review, 'translated_review': translated_review, 'sentiment': sentiment})
    sentiment_summary[place][sentiment] += 1

    print(f"Review for {place} has been added: '{translated_review}' -> Sentiment: {sentiment}")
    save_data()

## Fungsi untuk menampilkan ringkasan jumlah review dan sentimen untuk masing-masing tempat

In [ ]:
#Membuat fungsi untuk menampilkan rangkuman review dan sentimen setiap tempat
def display_summary():
    """
    Fungsi untuk menampilkan jumlah review per sentimen untuk masing-masing tempat.
    """
    print("\nSentiment Summary:")
    for place, sentiments in sentiment_summary.items():
        print(f"Place name: {place}")
        for sentiment, count in sentiments.items():
            print(f"  {sentiment}: {count} review") # Menampilkan jumlah dari masing-masing sentimen (positif, netral, dan negatif)
        print()

## Fungsi untuk menampilkan seluruh review yang tercatat untuk seluruh tempat

In [ ]:
#Membuat fungsi yang menampilkan seluruh review dari setiap tempat
def display_reviews():
    """
    Fungsi untuk menampilkan semua review untuk masing-masing tempat.
    """
    print("\nReview for each place:")
    #Mengambil nama tempat
    for place, reviews in places_reviews.items():
        print(f"\nReview for {place}:")
        #Mengambil review dan sentimen
        for data in reviews:
            print(f"  Review: {data['review']}")
            print(f"  Review (translated)): {data['translated_review']}")
            print(f"  Sentiment: {data['sentiment']}\n")

# Main menu untuk input dari user

In [ ]:
#Membuat fungsi untuk menu interaktif
def main():
    load_data()
    while True:
      #Menampilkan menu tersedia
        print("\nMenu:")
        print("1. Add review")
        print("2. View all review")
        print("3. Sentiment summary")
        print("4. Reset all review data")
        print("5. Exit")

        #Menerima masukan menu yang dipilih
        choice = input("Choose menu (1/2/3/4/5): ").strip()

        #Menentukan kondisi sesuai menu yang dipilih
        if choice == '1':
            add_review_interactive()
        elif choice == '2':
            display_reviews()
        elif choice == '3':
            display_summary()
        elif choice == '4':
            reset_history()
        elif choice == '5':
            print("Exit from program. Data will be saved.")
            save_data()
            break
        else:
            print("Invalid input. Please try again.")

# Menjalankan program
if __name__ == "__main__":
    main()

Data loaded successfully.

Menu:
1. Add review
2. View all review
3. Sentiment summary
4. Reset all review data
5. Exit
Choose menu (1/2/3/4/5): 1
Enter the place name: Alas Kedaton
Enter your review about the place (can be in any language): Sangat Bagus
Review for Alas Kedaton has been added: 'Very good' -> Sentiment: Positive
Data has been saved.

Menu:
1. Add review
2. View all review
3. Sentiment summary
4. Reset all review data
5. Exit
Choose menu (1/2/3/4/5): Alas Kedaton
Invalid input. Please try again.

Menu:
1. Add review
2. View all review
3. Sentiment summary
4. Reset all review data
5. Exit
Choose menu (1/2/3/4/5): Gakbanget
Invalid input. Please try again.

Menu:
1. Add review
2. View all review
3. Sentiment summary
4. Reset all review data
5. Exit
Choose menu (1/2/3/4/5): 1
Enter the place name: Alas Kedaton
Enter your review about the place (can be in any language): Gabanget
Review for Alas Kedaton has been added: 'Gabang' -> Sentiment: Netral
Data has been saved.

Menu: